In [1]:
import tensorflow as tf
import joblib

2024-08-21 19:11:09.220984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 19:11:12.697759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 19:11:12.703584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 19:11:20.604163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
hidden_weights = joblib.load("model/hidden_weights.pkl")
hidden_biases = joblib.load("model/hidden_biases.pkl")
output_weights = joblib.load("model/output_weights.pkl")
output_biases = joblib.load("model/output_biases.pkl")

In [4]:
model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [10]:
hidden_layer = model_cnn.layers[-2]
output_layer = model_cnn.layers[-1]

In [11]:
hidden_layer.set_weights((hidden_weights, hidden_biases))
output_layer.set_weights((output_weights, output_biases))

In [12]:
model_cnn.save("model/mnist_model_cnn")

INFO:tensorflow:Assets written to: model/mnist_model_cnn/assets


INFO:tensorflow:Assets written to: model/mnist_model_cnn/assets


In [13]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [20]:
y_train_pred = model_cnn.predict(X_train).argmax(axis=1)

1875/1875 [==============================] - 3s 2ms/step


In [21]:
y_test_pred = model_cnn.predict(X_test).argmax(axis=1)

313/313 [==============================] - 1s 2ms/step


In [18]:
from sklearn.metrics import accuracy_score

In [22]:
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

### Accuracy without scaling pixel values

In [23]:
print(f"{train_accuracy=}\n{test_accuracy=}")

train_accuracy=0.9518166666666666
test_accuracy=0.9581


### Scaling pixel values

In [24]:
X_train_scaled = X_train/255.
X_test_scaled = X_test/255.

In [25]:
y_train_scaled_pred = model_cnn.predict(X_train_scaled).argmax(axis=1)
y_test_scaled_pred = model_cnn.predict(X_test_scaled).argmax(axis=1)
train_scaled_accuracy = accuracy_score(y_train, y_train_scaled_pred)
test_scaled_accuracy = accuracy_score(y_test, y_test_scaled_pred)
print(f"{train_scaled_accuracy=}\n{test_scaled_accuracy=}")

2024-08-21 19:25:40.665837: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


313/313 [==============================] - 1s 2ms/step
train_scaled_accuracy=0.9875
test_scaled_accuracy=0.9901


### Hence, scale pixel values by dividing by 255 before passing to the model.